In [15]:
import os
import besos
from besos import eppy_funcs as ef
from besos import eplus_funcs as ep
from besos.problem import EPProblem
from besos.evaluator import EvaluatorEP
from besos.parameters import wwr, RangeParameter, FieldSelector, FilterSelector, GenericSelector, Parameter, expand_plist,DependentParameter
from besos.objectives import MeterReader,time_series_values,EPResults
from besos.optimizer import NSGAII, df_solution_to_solutions
import pandas as pd

In [2]:
# Open the IDF file
idf_directory = "/home/ict4bd/Desktop/ict4building/DesignBuilderModel/1_layer_window.idf"
weather_file = '/home/ict4bd/Desktop/ict4building/Requirements/CHE_Geneva.067000_IWEC/CHE_Geneva.067000_IWEC.epw'
building = ef.get_building(idf_directory)

In [4]:
# check the version
ep.get_idf_version(building)

'9.0'

In [9]:
# Set up evaluator
parameters = []
parameters = expand_plist( # Use helper function to make parameter list
    {'Building':                      # Name from IDF Building object
          {'North Axis':(0,359)}}     # Change orientation from North
    )
print(parameters)

[Parameter(name='North Axis', selector=FieldSelector(field_name='North Axis', object_name='Building'), value_descriptor=RangeParameter(min=0, max=359))]


In [10]:
objectives = ['Electricity:Facility','DistrictCooling:Facility', 'DistrictHeating:Facility']
besos_problem = EPProblem(parameters, objectives)

In [20]:
evaluator = EvaluatorEP(besos_problem, building, out_dir='outputdir', err_dir='outputdir') # outputdir must exist; E+ files will be written there
runs = pd.DataFrame.from_dict({'0': [0],'1':[90],'2':[180],'3':[270]}, orient='index') # Make a dataframe of runs with different oriantation
outputs = evaluator.df_apply(runs) # Run this as a test,
outputs

,Electricity:Facility,DistrictCooling:Facility,DistrictHeating:Facility
0,5.691270e+09,1.278309e+09,2.052586e+10
1,5.687283e+09,1.878403e+09,2.057794e+10
2,5.685346e+09,1.076052e+09,2.085595e+10
3,5.692108e+09,1.656532e+09,2.280406e+10


In [48]:
outputs['Total'] = 0
outputs['Total'] = outputs.sum(axis=1)  # axis = 0 is the default sum rows
outputs

,Electricity:Facility,DistrictCooling:Facility,DistrictHeating:Facility,Total
0,5.691270e+09,1.278309e+09,2.052586e+10,2.749544e+10
1,5.687283e+09,1.878403e+09,2.057794e+10,2.814363e+10
2,5.685346e+09,1.076052e+09,2.085595e+10,2.761735e+10
3,5.692108e+09,1.656532e+09,2.280406e+10,3.015270e+10


In [50]:
outputs[outputs['Total']==outputs['Total'].min()]

,Electricity:Facility,DistrictCooling:Facility,DistrictHeating:Facility,Total
0,5.691270e+09,1.278309e+09,2.052586e+10,2.749544e+10


In [ ]:
# the north axis = 0 is the best oriantation of the building